In [ ]:
import neptune.new as neptune
run = neptune.init(project='mila1/kaggle-335',
                   api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkMzdkYWQxNi1iYjlmLTRlODktOTZiZi1mYjY0MjkwM2FkYTYifQ==') # your credentials

run["JIRA"] = "NPT-952"
run["parameters"] = {"learning_rate": 0.001,
                     "optimizer": "Adam"}

for epoch in range(100):
   run["train/loss"].log(epoch * 0.4)
run["eval/f1_score"] = 0.66

n this model, we can control the regularization by means of the penalty and C parameters (inverse of regularization strength - smaller values specify stronger regularization) to deal with the overfitting:

In [ ]:
from sklearn.linear_model import LogisticRegression

m = LogisticRegression(
    penalty='l1',
    C=0.1
)
m.fit(train[labels], target)
m.predict_proba(test[labels])[:,1]

For tree-based models like XGBoost, we can control the overfitting by tuning a series of parameters:

Restricting the maximum depth of trees via max_depth (low values)
Making the model more conservative via gamma and eta (high values)
L1 and L2 regularization via reg_alpha and reg_lambda (high values)

In [ ]:
from xgboost import XGBClassifier

m = XGBClassifier(
    max_depth=2,
    gamma=2,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
)
m.fit(train[labels], target)
m.predict_proba(test[labels])[:,1]

### Stackig

In [ ]:
from mlxtend.classifier import StackingClassifier

m = StackingClassifier(
    classifiers=[
        LogisticRegression(),
        XGBClassifier(max_depth=2)
    ],
    use_probas=True,
    meta_classifier=LogisticRegression()
)

m.fit(train[labels], target),
preds['stack_pred'] = m.predict_proba(test[labels])[:,1]
preds.head()